# Load Test on MyAdroit (GPU nodes in the cloud)

In [4]:
%%time
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms, models
from torch.optim.lr_scheduler import StepLR

CPU times: user 26 µs, sys: 5 µs, total: 31 µs
Wall time: 32.2 µs


In [5]:
!nvidia-smi

Thu Mar 21 15:57:56 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 545.23.08              Driver Version: 545.23.08    CUDA Version: 12.3     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A10G                    On  | 00000000:00:1E.0 Off |                    0 |
|  0%   28C    P0              57W / 300W |   1452MiB / 23028MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
import os
from time import time
from datetime import datetime
start_dt = datetime.now()
start = time()

use_cuda = torch.cuda.is_available()

torch.manual_seed(42)
device = torch.device("cuda") if use_cuda else torch.device("cpu")

train_kwargs = {'batch_size': 192}
test_kwargs  = {'batch_size': 192}
if use_cuda:
    cuda_kwargs = {'num_workers': 4, 'pin_memory': True}
    train_kwargs.update(cuda_kwargs)
    test_kwargs.update(cuda_kwargs)

transform=transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize((0.1307,), (0.3081,))])
dataset1 = datasets.ImageFolder(root="/home/jdh4/cats_vs_dogs/training_set/", transform=transform)
dataset2 = datasets.ImageFolder(root="/home/jdh4/cats_vs_dogs/test_set/", transform=transform)

train_loader = torch.utils.data.DataLoader(dataset1, shuffle=True, **train_kwargs)
test_loader  = torch.utils.data.DataLoader(dataset2, shuffle=True, **test_kwargs)

print("Loading model ... ", end="", flush=True)
checkpoint = torch.load('/home/jdh4/.cache/torch/hub/checkpoints/resnet50-11ad3fa6.pth')
model = models.resnet50(pretrained=False)
model.load_state_dict(checkpoint)
print("done.")
for param in model.parameters():
    param.requires_grad = False
model.fc = nn.Sequential(nn.Linear(2048, 128), nn.ReLU(inplace=True), nn.Linear(128, 2))
model = model.to(device)
optimizer = optim.Adadelta(model.fc.parameters(), lr=1.0)

def train(model, device, train_loader, optimizer, epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(F.log_softmax(output, dim=1), target)
        loss.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print(f"epoch {epoch}, batch {batch_idx}")

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
    test_loss /= len(test_loader.dataset)

epochs = 3
scheduler = StepLR(optimizer, step_size=1, gamma=0.7)
for epoch in range(1, epochs + 1):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)
    scheduler.step()

print(f"{round(time() - start)} seconds, timestamp={start_dt}, {os.environ['USER']}, cuda={use_cuda}")

Loading model ... done.
epoch 1, batch 0
epoch 1, batch 10
epoch 1, batch 20
epoch 1, batch 30
epoch 1, batch 40
epoch 2, batch 0
